In [ ]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

In [ ]:
conf=SparkConf()
conf.set("spark.executor.memory", "2g")
conf.set("spark.driver.memory", "12g")
conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

In [ ]:
class MatchPruner:
    def __init__(self, mdw: DataFrame, match_suggestion: DataFrame, config: dict) -> None:
        self.mdw = mdw
        self.match_suggestion = match_suggestion
        self.config = config
        
    def preprocessing_matches(self, matches: DataFrame):
        matches = matches.withColumn("seller_type", lower(col("seller_type")))\
            .withColumn("match_type", when( col("match_type").isNotNull(), lower(col("match_type")) ).otherwise(col("match_type")) )
        return matches
    
    def prune_1p_3p_matches(self, match_suggestion: DataFrame):
        filtered_matches = None
        pruned_matches = None
        if self.config['seller_type'] == '1p':
            filtered_matches = match_suggestion.filter(col('seller_type') == '1p')
            pruned_matches = match_suggestion.filter(col('seller_type') == '3p')
            pruned_matches = pruned_matches.withColumn("STATUS", lit("pruned_by_seller_type_1p"))
        elif self.config['seller_type'] == '1p':
            filtered_matches = match_suggestion.filter(col('seller_type') == '3p')
            pruned_matches = match_suggestion.filter(col('seller_type') == '1p')
            pruned_matches = pruned_matches.withColumn("STATUS", lit("pruned_by_seller_type_3p"))
        else:
            filtered_matches = match_suggestion
        return filtered_matches, pruned_matches
    
    def separate_audited_unaudited_base_product_matches(self, match_suggestion:DataFrame, mdw:DataFrame):
        unaudited_base_product_matches = match_suggestion.join(mdw, 'base_sku_uuid', 'left_anti')
        audited_base_product_matches = match_suggestion.join(mdw, 'base_sku_uuid', 'inner')
        return unaudited_base_product_matches, audited_base_product_matches
    
    def separate_audited_and_unaudited_matches(self, audited_base_product_matches:DataFrame, mdw:DataFrame):
        audited_matches = audited_base_product_matches.join(mdw, 'pair_id', 'inner')
        unaudited_matches = audited_base_product_matches.join(mdw, 'pair_id', 'left_anti')
        return audited_matches, unaudited_matches
    
    def 
        
    